#  Introdução ao Python
## Professor: Luiz Ferreira

## Módulo 2

### Python para Web

_____________

## Editando os arquivos editar_cliente, listar_cliente e criar_cliente

Vamos trabalhar o conteúdo das páginas criadas.

Cada arquivo deverá fazer o que sugere seu nome, dito isso, vamos começar pelo arquivo "listar_cliente.html". Mas antes, devemos criar o modelo de nosso banco de dados, o responsável pelas informações do que será salvo.

Para isso, vamos editar o nosso arquivo "model.py" dentro de principal, de forma que fique semelhante a :

In [ ]:
from django.db import models
# Create your models here.


class clientes(models.Model):
    nome = models.CharField(max_length=400)
    telefone = models.CharField(max_length=50)
    email = models.CharField(max_length=120)


Com o model criado, podemos chamá-lo em nossas outras aplicações, permitindo a criação e manipulação da base de dados.

Vamos precisar criar as entradas no arquivo "url.py" para que possamos acessar nossas páginas, de forma que fique semelhante a:

In [ ]:
from django.urls import path
from principal import views as principal_views


urlpatterns = [
    path('', principal_views.principal, name="principal"),
    path('listar', principal_views.listar_cliente, name="listar"),
    path('novo', principal_views.criar, name="criar"),
    path('delete/<int:id>', principal_views.delete, name='delete'),
    path('editar/<int:id>', principal_views.editar, name='editar'),
]

Um detalhe importante é a inserção da tag < int:id >, que irá receber por parâmetro, via método POST, o valor do id a ser modificado ou apagado.

### GET e POST

Sendo informal e direto, o HTTP – Hypertext Transfer Protocol(Protocolo de Transferência de Dados) é um protocolo ou padrão de rede implementado em cima do TCP para que browsers e servidores possam se comunicar.

Um cliente HTTP é um browser, ou seja, Chrome, Firefox, Internet Explore, entre outros, são software’s que se comunicam com servidores através do HTTP. O tipo de servidor a que me refiro aqui também são software’s, como por exemplo, o Apache e o IIS(Microsoft).

Os principais métodos de comunicação HTTP são: GET e POST. Podemos diferenciá-las como:

- Visibilidade – A grande diferença entre os métodos GET e POST provavelmente é a visibilidade. Uma requisição GET é enviada como string anexada a URL, enquanto que a requisição POST é encapsulada junto ao corpo da requisição HTTP e não pode ser vista.

- Tamanho – Como a requisição GET é feita via URL, obviamente há uma limitação no tamanho da mensagem enviada. A string não pode conter mais que 255 caracteres(embora exista diferenças entre navegadores, mas em geral o limite é 255). Já na requisição POST não há limitações de comprimento da mensagem, já que a mesma é enviada no corpo da requisição HTTP.

- Performance – A requisição GET é relativamente mais rápida, já que ela é mais simples. Na requisição POST há uma perda de tempo no encapsulamento da mensagem.

- Tipos – Já que GET é enviado via URL, então nós sabemos que ela só transporta textos. A requisição POST não tem restrições, pode transportar tanto texto, como dados binários.

- Favoritos/Bookmarks – Por se tratar apenas de uma URL, a requisição GET pode ser armazenada em cache, ou em um sistema de bookmark(favoritos). A mesma coisa não é possível para requisições POST.

- Método HTML Padrão – GET é o método HTML padrão. Para submeter um formulário HTML usando POST é preciso especificar no atributo “method” o valor “POST”.

- Dados – As requisições GET são limitadas ao padrão ASCII, enquanto que requisições POST também podem usar o atributo “enctype” com o valor “multipart/form-data”, que faz uso do padrão UCS(Universal Multiple-Octet Coded Character Set).

### Continuando ... A edição do arquivo views.py

Para que o valor seja recebido corretamente para edição ou exclusão, a lista chegar corretamente para a exibição e para realizar o armazenamento de um novo registro, devemos editar nossas views, para que elas entendam o que necessitamos:

- Vamos começar: como iremos trabalhar com o salvamento de um formulário, para que possamos utilizá-lo de forma eficiente, devemos criar uma classe chamada de Meta que irá definir padrões personalizados para a associar cada elemento a um valor quando trabalhado em formulários ela é um guia de qual model utilizar e quais campos processar.

In [ ]:
class ClienteForm(ModelForm):
    class Meta:
        model = clientes
        fields = ['id', 'nome', 'telefone', 'email']

Com a classe ClienteForm informamos ao nosso app que existe um modelo a ser seguido quando trabalharmos com um formulário de cliente, apontando características como: model relacionado ao formulário e quais campos ele irá encontrar tanto na base quanto no model para trabalharmos.

In [ ]:
def listar_cliente(request):
    cliente = clientes.objects.all()
    context = {'cliente': cliente}
    return render(request, 'listar_cliente.html', context)

A função listar_cliente será a responsável pela página listar_cliente, com a URL /listar. 

Dito isso, essa será a forma com que iremos recuperar os dados em uma lista. Vejamos:

- a variável cliente receberá todos os objetos encontrados no banco de dados clientes
- context será o responsável por tratar todos os objetos de forma com que sejam acessíveis uma a uma
- o return será padrão, retornará uma requisição HTML cujo endereço é listar_cliente.html, porém, com o adicional da lista contida em context

A função delete será a responsável por eliminar um registro de nossa base de dados.

Observemos: 

- cliente recuperará um cliente com a variável id específica, que será informada quando do clique no botão APAGAR na página listar_cliente, com o clique ele obterá a informação do id e passará à exclusão
- cliente.delete() realizará a remoção
- return redirect fará com que, após a exclusão, redirecione o usuário para uma página específica, nesse caso, ele realizará a atualização da página listar_clientes


In [ ]:
def delete(request, id):
    cliente = clientes.objects.get(id=id)
    cliente.delete()
    return redirect('/listar')

A função delete será a responsável por eliminar um registro de nossa base de dados.

Observemos: 

- cliente recuperará um cliente com a variável id específica, que será informada quando do clique no botão APAGAR na página listar_cliente, com o clique ele obterá a informação do id e passará à exclusão
- cliente.delete() realizará a remoção
- return redirect fará com que, após a exclusão, redirecione o usuário para uma página específica, nesse caso, ele realizará a atualização da página listar_clientes


In [ ]:
def editar(request, id):
    post = get_object_or_404(clientes, pk=id)
    form = ClienteForm(request.POST or None, instance=post)
    if form.is_valid():
        form.save()
        return redirect('/listar')
    return render(request, 'editar_cliente.html', {'form': form})


def criar(request):
    form = ClienteForm(request.POST or None)
    if form.is_valid():
        form.save()
        return redirect('/listar')
    return render(request, 'criar_cliente.html', {'form': form})

As funções editar e criar são muito semelhantes quanto ao seu funcionamento, a única características que o diferem é o uso do mesmo mecanismo de utilização da variável id pela função delete.

Observemos:

- uma variável post que irá procurar dentro de todos os clientes o que possui o id idêntico ao passado durante o clique
- a variável form irá utilizar o MetaClass em que informamos como trabalharemos esses formulários que manipularão os dados de clientes
- o if form.is_valid fará uma checagem sobre os dados apresentados pelo formulário e como deveriam ser (not null, string ou numérico) ...
- o form.save() irá executar o commit dos dados digitados no formulário

In [ ]:
def principal(request):
    return render(request, 'principal.html')

A função principal, irá continuar sendo mínima, apenas solicitando uma requisição HTML com a página principal.


#### Com as views contruídas e a URL configurada corretamente, procederemos ao conteúdo das páginas criadas

### listar_clientes.html

In [ ]:
{% extends 'base.html' %}
{% load static %}

{% block body %}

<div class="container">
    <br><br>
    <table class="table centered table-bordered">
        <thead class="alert-warning">
            <tr>
                <th>Nome</th>
                <th>Telefone</th>
                <th>Email</th>
                <th>Ações</th>
            </tr>
        </thead>
        <tbody>
            {% for cli in cliente %}
            <tr>
                <td>{{ cli.nome }}</td>
                <td>{{ cli.telefone }}</td>
                <td>{{ cli.email }}</td>
                <td>
                    <a class="waves-effect waves-light btn orange" href="{% url 'editar' cli.id %}">
                        <span class="glyphicon glyphicon-edit"></span> Editar</a>
                    <a class="waves-effect waves-light btn red" href="{% url 'delete' cli.id %}">
                        <span class="glyphicon glyphicon-trash"></span> Apagar</a>
                </td>
            </tr>
            {% endfor %}
        </tbody>
    </table>
</div>
<div center class="container center-align">
    <a class="waves-effect waves-light btn green" href="{% url 'criar' %}"><span class="glyphicon glyphicon-trash"></span> Novo Cliente</a>
    <br><br>
</div>
{% endblock %}

Podemos ver aqui a codificação em HTML da página, chamando a atenção para o uso do arquivo `base` e o uso de codificação em Python dentro da página, como vemos no {% for cli in cliente %}, que irá fazer a manipualação da base de dados, e {{ cli.nome }} que irá manipular cada célula de nossa tabela de dados.

### editar.html

In [ ]:
{% extends 'base.html' %}
{% load static %}

{% block body %}
<div class="container">
    <h4>Editando cliente</h4>
    <form method="post">{% csrf_token %}
        {{ form.as_p }}
        <button class="btn waves-effect waves-light" type="submit" name="action">Submit<i class="material-icons right">send</i></button>
    </form>
</div>
{% endblock %}

Aqui vemos novamente a codificação em HTML utilizando o arquivo `base` e notamos a construção do formulário.

- {% csrf_token %} =  Essa tag é a responsável por implementar o middleware de CSRF que é uma forma fácil de implementar soluções contra ataques de pessoas mal-intencionadas, como por exemplo, ataques de Cross Site quando um botão responsável por uma ação é executado usando as credenciais de outro usuário. Esse tipo de erro comum força o navegador de um usuário a trabalhar de forma errada, que evitamos simplesmente usando essa tag. A primeira defesa contra ataques de CSRF é garantir que as solicitações GET (e outros métodos "seguros", conforme definido pela RFC 7231 # seção-4.2.1) sejam livres de efeitos colaterais. Os pedidos através de métodos "inseguros", como POST, PUT e DELETE, podem ser protegidos seguindo as etapas abaixo.
- {{ form.as_p }} = Implementa um formulário com validação construído com base no Meta do ModelForm implementado anteriormente.

### criar_cliente.html

In [ ]:
{% block body %}
<div class="container">
    <h4>Cadastrar cliente</h4>
    <ul>
        <form method="post">{% csrf_token %}
            {{ form.as_p }}
            <button class="btn waves-effect waves-light" type="submit" name="action">Submit
                <i class="material-icons right">send</i>
            </button>
        </form>
    </ul>
</div>
{% endblock %}

Conforme vimos acima, utilizamos o csrf_token para garantir a segurança de nosso formulário e o form.as_p para criar nosso formulário, muito semelhante ao funcionamento do `editar_cliente.html`.